In [ ]:
import torch
from torchvision import datasets, transforms
mnist_train = datasets.MNIST(
    root='/workspaces/cosc470/data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

mnist_test = datasets.MNIST(
    root='/workspaces/cosc470/data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

mnist_train.data.#



100%|██████████| 9.91M/9.91M [00:00<00:00, 141MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 6.47MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 45.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 11.6MB/s]


In [13]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(20 * 12 * 12, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        # relu function equation: z = Summation(w * x) + b (some form of this)
        # softmax function equation: softmax(z) =  1 / 1+e^(z) (some form of this as well)
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 20 * 12 * 12)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

#model = SimpleCNN()
bad_prediction = model.forward(torch.randn(1,1,28,28))
print(bad_prediction)
real_image = mnist_train[0][0].unsqueeze(0)
good_prediction = model.forward(real_image)
print(good_prediction)



tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.3887e-44, 8.6894e-22,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00]],
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 9.2494e-37, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],
       grad_fn=<SoftmaxBackward0>)


In [11]:
batch_size = 10
learning_rate = 0.001
epochs = 20

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


for epoch in range(20):
    running_loss = 0.0
    for images, labels in train_loader:
        #Foward
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    correct = 0
    for testimage in mnist_test:
        a = testimage[0].unsqueeze(0)
        a = model.forward(a)
        if torch.argmax(a) == testimage[1]:
            correct += 1
    print(f"Test Accuracy: {100 * correct / len(mnist_test):.2f}%")


Epoch [1/20], Loss: 1.4678
Test Accuracy: 98.51%
Epoch [2/20], Loss: 1.4676
Test Accuracy: 98.00%
Epoch [3/20], Loss: 1.4677
Test Accuracy: 98.75%
Epoch [4/20], Loss: 1.4675
Test Accuracy: 98.64%
Epoch [5/20], Loss: 1.4675
Test Accuracy: 98.85%
Epoch [6/20], Loss: 1.4667
Test Accuracy: 98.77%
Epoch [7/20], Loss: 1.4665
Test Accuracy: 98.53%
Epoch [8/20], Loss: 1.4666
Test Accuracy: 98.77%
Epoch [9/20], Loss: 1.4670
Test Accuracy: 98.76%
Epoch [10/20], Loss: 1.4672
Test Accuracy: 98.61%
Epoch [11/20], Loss: 1.4668
Test Accuracy: 98.85%
Epoch [12/20], Loss: 1.4663
Test Accuracy: 98.67%
Epoch [13/20], Loss: 1.4662
Test Accuracy: 98.83%
Epoch [14/20], Loss: 1.4659
Test Accuracy: 98.84%
Epoch [15/20], Loss: 1.4667
Test Accuracy: 98.80%
Epoch [16/20], Loss: 1.4665
Test Accuracy: 98.79%


KeyboardInterrupt: 